In [ ]:
from analysis import *
from preprocess import *
from utils import *
from train import *

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

if torch.cuda.is_available():
    torch.set_default_device('cuda')

## Load data and preprocessing

1. Load train data

In [ ]:
train_set = pd.read_csv('./datasets/digit-recognizer/train.csv')
train_set = train_set.to_numpy()
np.random.shuffle(train_set)

2. Preprocessing

In [ ]:
train_x = []
for i in range(len(train_set)):
    train_x.append(np.array(train_set[i, 1:]).reshape(1, 28, 28).astype(np.float32))

train_x = np.array(train_x)
train_y = train_set[:, 0]

print(train_x.shape, train_y.shape)

## Design model and validation

1. Model design

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, padding=2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        
        self.batch_norm2 = nn.BatchNorm2d(64)

        self.fc1 = nn.Linear(64*7*7, 2048)
        self.fc2 = nn.Linear(2048, 1024)
        self.fc3 = nn.Linear(1024, 256)
        self.fc4 = nn.Linear(256, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)

        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2)

        x = x.view(-1, 64*7*7)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.dropout(x, 0.1)
        x = F.relu(self.fc3(x))
        x = F.dropout(x, 0.1)
        x = self.fc4(x)
        return x

2. Split to train and validation data

In [ ]:
train_x = torch.tensor(train_x)
train_y = torch.tensor(train_y)

train_x, train_y, valid_x, valid_y = train_valid_split(train_x, train_y, 0.2)
train_dataloader = make_dataloader(train_x, train_y, 64)
valid_dataloader = make_dataloader(valid_x, valid_y, 64)

3. Make instance

In [ ]:
model = CNN()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

trainer = Trainer(model, optimizer, criterion, 'classification')

4. train with validation

In [ ]:
augment_transform = augment_transformer(False, False, 15)

trainer.train(10, train_dataloader, valid_dataloader, augment_transformer=augment_transform, augment_ratio=0.3, verbose=True)

## Predict test data and submission

1. Load test data and preprocessing

In [ ]:
test_set = pd.read_csv('./datasets/digit-recognizer/test.csv')

test_set = test_set.to_numpy()
test_x = []
for i in range(len(test_set)):
    test_x.append(np.array(test_set[i, :]).reshape(1, 28, 28).astype(np.float32))

test_x = np.array(test_x)
test_x = torch.tensor(test_x)

2. Train model with all train data

In [ ]:
train_x = []
for i in range(len(train_set)):
    train_x.append(np.array(train_set[i, 1:]).reshape(1, 28, 28).astype(np.float32))

train_x = np.array(train_x)
train_y = train_set[:, 0]

train_x = torch.tensor(train_x)
train_y = torch.tensor(train_y)

train_dataloader2 = make_dataloader(train_x, train_y, 64)

In [ ]:
model_real = CNN()
optimizer_real = optim.Adam(model_real.parameters(), lr=0.0001)
criterion_real = nn.CrossEntropyLoss()

trainer_real = Trainer(model_real, optimizer_real, criterion_real, 'classification')

augment_transform = augment_transformer(False, False, 15)
trainer_real.train(3, train_dataloader2, None, augment_transformer=augment_transform, augment_ratio=0.3, verbose=True)

3. Convert model to evaluation mode and test

In [ ]:
model_real.eval()
testloader = make_dataloader(test_x, None, 256)

pred = trainer_real.test(testloader)

3. Save predict output

In [ ]:
submission_csv = pd.DataFrame({'ImageId': np.arange(1, len(pred)+1), 'Label': pred})
submission_csv.to_csv('./datasets/digit-recognizer/submission.csv', index=False)